In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from Utils import Utils
from Constants import Constants
from Propensity_socre_network import Propensity_socre_network
from Utils import Utils
from PS_Manager import PS_Manager
from PS_Treated_Generator import PS_Treated_Generator

from GAN import Generator, Discriminator
from sklearn.neighbors import NearestNeighbors

from GAN_Manager import GAN_Manager
from Utils import Utils

from matplotlib import pyplot
from torch.autograd.variable import Variable
from collections import OrderedDict
from scipy.special import expit

import os
from os.path import join
import sys
from dataloader import DataLoader
from DCN_Experiments import DCN_Experiments
from Metrics import Metrics

In [2]:
def __get_ps_model(ps_model_type, iter_id,
                   input_nodes, device,
                   np_covariates_X_train, 
                   np_covariates_X_test, 
                   np_covariates_Y_train,
                   np_covariates_Y_test, dL):
    ps_train_set = dL.convert_to_tensor(np_covariates_X_train, np_covariates_Y_train)
    ps_test_set = dL.convert_to_tensor(np_covariates_X_test,
                                            np_covariates_Y_test)
    ps_manager = PS_Manager()
    if ps_model_type == Constants.PS_MODEL_NN:
        return ps_manager.get_propensity_scores(ps_train_set,
                                                    ps_test_set, iter_id,
                                                    input_nodes, device)

In [3]:
iter_id = 1

In [4]:
input_nodes = 17
device = Utils.get_device()
ps_model_type=Constants.PS_MODEL_NN
train_path = "Dataset/jobs_DW_bin.new.10.train.npz"
test_path = "Dataset/jobs_DW_bin.new.10.test.npz"
split_size = 0.8
dL = DataLoader()

Constants.PROP_SCORE_NN_EPOCHS

150

In [5]:
np_covariates_X_train, np_covariates_X_test, np_covariates_T_train, \
        np_covariates_T_test \
            = dL.load_train_test_jobs(train_path, test_path, iter_id)

Numpy Train Statistics:
(2570, 19)
(2570, 1)
 Numpy Test Statistics:
(642, 19)
(642, 1)


In [6]:
ps_score_list_train, ps_score_list_test, ps_model = __get_ps_model(ps_model_type, 
                                                                   1, input_nodes,
                                                                device, 
                                                                np_covariates_X_train, 
                                                                np_covariates_X_test, 
                                                                np_covariates_T_train,
                                                                np_covariates_T_test,
                                                                dL)


############### Propensity Score neural net Training ###############
.. PS Training started ..
Epoch: 50, loss: 13.957078225910664, correct: 2347/2570, accuracy: 0.9132295719844358
Epoch: 100, loss: 12.979373145848513, correct: 2371/2570, accuracy: 0.9225680933852141
Epoch: 150, loss: 12.010436907410622, correct: 2381/2570, accuracy: 0.9264591439688716
Training Completed..


In [7]:
print(len(ps_score_list_train))
print(len(ps_score_list_test))

2570
642


In [8]:
print("--->>Train size: ")
data_loader_dict_train = dL.prepare_tensor_for_DCN(np_covariates_X_train,
                                                        np_covariates_T_train,
                                                        ps_score_list_train,
                                                        False)
print("--->>Test size: ")
data_loader_dict_test = dL.prepare_tensor_for_DCN(np_covariates_X_test,
                                                      np_covariates_T_test,
                                                      ps_score_list_test,
                                                      False)
print(len(data_loader_dict_train["treated_data"]))

tensor_treated_train_original = \
                Utils.create_tensors_from_tuple(data_loader_dict_train["treated_data"])
tensor_control_train_original = \
                Utils.create_tensors_from_tuple(data_loader_dict_test["control_data"])

--->>Train size: 
Big X: (2570, 21)
 Treated Statistics ==>
(237, 17)
 Control Statistics ==>
(2333, 17)
--->>Test size: 
Big X: (642, 21)
 Treated Statistics ==>
(60, 17)
 Control Statistics ==>
(582, 17)
4


In [9]:
ps_t = PS_Treated_Generator(data_loader_dict_train, ps_model)
balanced_dataset_dict = ps_t.simulate_treated_semi_supervised(input_nodes, iter_id, device)

-> Matched Control: (237, 17)
-> UnMatched Control: (2205, 17)
-> GAN training started
Epoch: 1000, D_loss: 32.7128946185112, D_score_real: 23.75793957710266, D_score_Fake: 11.37055292725563, G_loss: 41.03593552112579, Prop_loss: 110.76152908802032
Epoch: 2000, D_loss: 30.848858952522278, D_score_real: 24.5241539478302, D_score_Fake: 10.49499523639679, G_loss: 44.557498812675476, Prop_loss: 61.76901459693909
Epoch: 3000, D_loss: 32.90187919139862, D_score_real: 23.768618285655975, D_score_Fake: 11.267085611820221, G_loss: 42.22055399417877, Prop_loss: 81.71781003475189
Epoch: 4000, D_loss: 31.761606335639954, D_score_real: 24.31711834669113, D_score_Fake: 10.861313104629517, G_loss: 42.7986261844635, Prop_loss: 51.57673639059067
Epoch: 5000, D_loss: 31.677600860595703, D_score_real: 24.180280685424805, D_score_Fake: 10.598519057035446, G_loss: 44.0903936624527, Prop_loss: 46.65553414821625
Epoch: 6000, D_loss: 30.43159329891205, D_score_real: 24.641621947288513, D_score_Fake: 10.398718

<Figure size 432x288 with 0 Axes>

In [10]:
tensor_treated_balanced_dcn = balanced_dataset_dict["tensor_treated_balanced_dcn"]
tensor_control_balanced_dcn = balanced_dataset_dict["tensor_control_balanced_dcn"]
dcn_experiments = DCN_Experiments(input_nodes, device)
dcn_pd_models_eval_dict = dcn_experiments.evaluate_DCN_Model(tensor_treated_train_original,
                                                                         tensor_control_train_original,
                                                                         tensor_treated_balanced_dcn,
                                                                         tensor_control_balanced_dcn,
                                                                         data_loader_dict_test)

----------------------------------------
###### Model 1: DCN - PD Supervised Training started ######
Train_mode: train_PD
epoch: 100, Treated + Control loss: 100.9043382124537
epoch: 200, Treated + Control loss: 79.30706841783076
epoch: 300, Treated + Control loss: 69.57941165697125
epoch: 400, Treated + Control loss: 58.21852614075187
----------------------------------------
###### Model 2: DCN PM GAN - No dropout - Supervised Training started ######
Train_mode: train_with_no_dropout
epoch: 100, Treated + Control loss: 378.2583334859115
epoch: 200, Treated + Control loss: 276.443437486341
epoch: 300, Treated + Control loss: 212.0829366477054
epoch: 400, Treated + Control loss: 211.10518839375973
###### Model 3: DCN PM GAN - Probability 0.2 - Supervised Training started ######
Train_mode: train_constant_dropout_2
epoch: 100, Treated + Control loss: 527.9360118243728
epoch: 200, Treated + Control loss: 485.9896758321023
epoch: 300, Treated + Control loss: 437.57827957144605
epoch: 400, 

In [18]:
from DCN_Model import DCN_shared, DCN_Y1, DCN_Y0
dcn_shared = DCN_shared(input_nodes=25).to(device)
dcn_y1 = DCN_Y1().to(device)
dcn_y0 = DCN_Y0().to(device)

for parameter in dcn_shared.parameters():
    print(parameter)

Parameter containing:
tensor([[ 0.0317, -0.1478,  0.0222,  ...,  0.0758, -0.0360, -0.0245],
        [-0.0277, -0.1564, -0.0991,  ..., -0.0930,  0.0260, -0.1200],
        [ 0.0654,  0.0289,  0.1126,  ..., -0.0465,  0.0835, -0.1509],
        ...,
        [-0.0780,  0.0008,  0.1628,  ...,  0.0586, -0.0519,  0.0502],
        [ 0.1067,  0.0452, -0.0452,  ..., -0.0565,  0.1068, -0.0332],
        [-0.1399,  0.1375,  0.1203,  ..., -0.0809,  0.1374,  0.0413]],
       requires_grad=True)
Parameter containing:
tensor([ 0.1870,  0.0983,  0.1168,  0.0631,  0.1718, -0.0091,  0.0595, -0.0551,
        -0.1238,  0.1107, -0.0758,  0.0302,  0.0383, -0.0222,  0.0544,  0.0318,
        -0.1370, -0.1065, -0.1296,  0.1576,  0.0743, -0.0758,  0.0382, -0.1077,
         0.1074,  0.0248, -0.1778,  0.0737, -0.1196,  0.0673,  0.0539, -0.1682,
        -0.0395, -0.1950, -0.0167,  0.1444,  0.1513,  0.1486, -0.1827, -0.1228,
         0.0948, -0.1344,  0.0894,  0.0451, -0.1720, -0.0920,  0.0160,  0.1927,
         0.1764

In [21]:
model_shared_path = "/Users/shantanughosh/Desktop/Shantanu_MS/Research/Mattia_Prosperi/PM_GAN/Propensity_Match_using_GAN/IHDP_Shalit/Models/DCN_PD_05/DCN_PD_02_shared_iter_0.pth"
dcn_shared.load_state_dict(torch.load(model_shared_path,
                                                   map_location=device))

for parameter in dcn_shared.parameters():
    print(parameter)
      

Parameter containing:
tensor([[-0.0461, -0.0044,  0.0336,  ..., -0.0382,  0.1260,  0.1600],
        [-0.1341,  0.0016, -0.0353,  ..., -0.0480,  0.0025,  0.1421],
        [ 0.1269,  0.1143,  0.1383,  ...,  0.0299,  0.1697, -0.0933],
        ...,
        [-0.0680,  0.1063, -0.1270,  ...,  0.0041,  0.0787,  0.0288],
        [-0.1253,  0.0152, -0.0296,  ...,  0.0797,  0.1280, -0.0515],
        [-0.1011, -0.1071, -0.0009,  ...,  0.1490,  0.0090,  0.0895]],
       requires_grad=True)
Parameter containing:
tensor([ 6.7925e-02, -1.6477e-01,  1.7984e-01, -1.3997e-01,  1.1499e-01,
         1.1982e-01, -1.2948e-01, -1.8051e-01,  6.7678e-03, -2.9903e-02,
        -1.7845e-01,  9.2904e-02, -7.1047e-02, -3.0446e-02,  1.7157e-01,
        -4.8873e-02, -1.1996e-01,  2.6273e-02, -7.5008e-02,  9.0494e-02,
         1.9212e-02, -2.1807e-02, -1.2082e-01, -1.6460e-01, -7.1176e-02,
         8.3949e-02,  1.3019e-01,  1.9206e-01, -1.5031e-01,  1.7718e-01,
         1.3786e-01, -7.5065e-02, -4.1751e-02,  7.0502e-0

In [31]:
a = torch.tensor(0.0000000000000000000000001)
print(a)

torch.log2(a)


tensor(1.0000e-25)


tensor(-83.0482)

In [22]:
model_shared_path = "/Users/shantanughosh/Desktop/Shantanu_MS/Research/Mattia_Prosperi/PM_GAN/Propensity_Match_using_GAN/IHDP_Shalit/Models/DCN_PD_02/DCN_PD_02_shared_iter_0.pth"
dcn_shared.load_state_dict(torch.load(model_shared_path,
                                                   map_location=device))

for parameter in dcn_shared.parameters():
    print(parameter)
      

Parameter containing:
tensor([[-0.1506, -0.1440,  0.0276,  ..., -0.0313, -0.0697, -0.0671],
        [ 0.0756,  0.0222,  0.0510,  ...,  0.0218, -0.0125, -0.0189],
        [ 0.0860,  0.0321, -0.1527,  ..., -0.0331,  0.1191, -0.0823],
        ...,
        [ 0.0326, -0.0673,  0.0588,  ..., -0.0569,  0.0317,  0.0207],
        [-0.0500,  0.1475, -0.0679,  ...,  0.0441, -0.0327,  0.0132],
        [ 0.1623, -0.1148,  0.0233,  ...,  0.1275,  0.1122,  0.1563]],
       requires_grad=True)
Parameter containing:
tensor([-0.1457, -0.1165,  0.0805, -0.1951, -0.1797, -0.0927,  0.1628, -0.1224,
         0.1297, -0.1875, -0.1158,  0.0419, -0.1956,  0.0041, -0.0251,  0.1110,
        -0.1024,  0.0313,  0.0760, -0.1495,  0.0124,  0.1586, -0.0959,  0.1061,
         0.1557,  0.1582, -0.1027, -0.0538, -0.0813, -0.0823,  0.1432,  0.1902,
        -0.0487, -0.0234, -0.1028,  0.1299,  0.2128,  0.1395, -0.1132, -0.1348,
         0.0958, -0.0970,  0.0205, -0.0323,  0.1878,  0.0120,  0.0051,  0.1188,
         0.0738